In [1]:
import collections
import dask
import gcsfs
import h5py
import io
import numba
import numpy as np
import os
import os.path
import pickle
import requests

from dask.distributed import Client
from dask_kubernetes import KubeCluster
from numba.typed import Dict

In [2]:
cluster = KubeCluster.from_yaml('worker-spec.yml')
cluster

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:    tcp://10.36.0.37:42901
distributed.scheduler - INFO -   dashboard at:                     :8787


In [3]:
client = Client(cluster)

distributed.scheduler - INFO - Receive client connection: Client-8acd23e8-389d-11ea-810f-26996f775df6
distributed.core - INFO - Starting established connection


Check that the client is working well.

In [32]:
@dask.delayed
def the_sum(a, b):
    return a + b
the_sum(the_sum(1, 2), 3).compute()

6

In [ ]:
if not os.path.exists('../../.gcs_tokens'):
    # Get a token
    gcsfs.GCSFileSystem(project='neuron-jungle', token='browser')

In [4]:
with open('../.gcs_tokens', 'rb') as f:
    credentials = pickle.load(f)
credentials = credentials[list(credentials.keys())[0]]
fs = gcsfs.GCSFileSystem(project='neuron-jungle', token=credentials)
fs.ls('l4dense/segmentation-volume')[:10]

['l4dense/segmentation-volume/x3y8z3.hdf5',
 'l4dense/segmentation-volume/x2y6z3.hdf5',
 'l4dense/segmentation-volume/x5y2z1.hdf5',
 'l4dense/segmentation-volume/x3y4z0.hdf5',
 'l4dense/segmentation-volume/x3y8z2.hdf5',
 'l4dense/segmentation-volume/x3y4z2.hdf5',
 'l4dense/segmentation-volume/x4y1z2.hdf5',
 'l4dense/segmentation-volume/x0y5z3.hdf5',
 'l4dense/segmentation-volume/x3y2z2.hdf5',
 'l4dense/segmentation-volume/x0y3z2.hdf5']

# Create a map from segment id to neuron id

In [14]:
def locally_cache(filename, credentials):
    fs = gcsfs.GCSFileSystem(project='neuron-jungle', token=credentials)
    with fs.open(f'l4dense/{filename}', 'rb') as f:
        data = f.read()
    with open(f'../cache/{filename}', 'wb') as f:
        f.write(data)
    return len(data)

locally_cache('dendrites.hdf5', credentials)

263193540

In [16]:
dendrites['dendrites']['agglomerate']

<HDF5 group "/dendrites/agglomerate" (11400 members)>

In [19]:
dendrites = h5py.File('../cache/dendrites.hdf5', 'r')

d = collections.defaultdict(lambda: [])
neuronId = np.array(dendrites['dendrites']['neuronId'])
for i, id in enumerate(neuronId):
    if id > 0:
        # Append the dendrite ids to the right slot.
        d[id] += np.array(dendrites['dendrites']['agglomerate'][str(i + 1)]).tolist()
    
neuron_map = {}
for neuron_id, segment_ids in d.items():
    for segment_id in segment_ids:
        neuron_map[segment_id] = neuron_id

In [6]:
@numba.jit(nopython=True)
def remap(data, the_map):
    b = np.zeros_like(data)
    for i in range(len(data)):
        if data[i] in the_map:
            b[i] = the_map[data[i]]
    return b

In [7]:
# To repaint: map dendrite ids to neuron id (default to 0)
@dask.delayed
def repaint(filename, credentials):    
    # Create a typed map for segment_to_neuron
    fs = gcsfs.GCSFileSystem(project='neuron-jungle', token=credentials)
    with fs.open('l4dense/neuron-map.pkl', 'rb') as f:
        segment_to_neuron = pickle.loads(f.read())

    the_map_typed = Dict.empty(key_type=numba.int32, value_type=numba.uint8)
    for k, v in segment_to_neuron.items():
        the_map_typed[k] = v
    
    with fs.open(f'l4dense/segmentation-volume/{filename}', 'rb') as f:    
        cube = h5py.File(f, 'r')
        
        a = np.zeros((1024, 1024, 1024), dtype=np.uint8)
        
        slice_size = 32
        nslices = int(1024 / slice_size)
        
        for j in range(nslices):
            subd = np.array(cube['data'][(slice_size*j):(slice_size*(j+1)), :, :])
            for i in range(nslices):
                a[i + j*slice_size, :, :] = remap(subd[i, :, :].ravel(), 
                                   the_map_typed).astype(np.uint8).reshape((1024, 1024))
    
    bio = io.BytesIO()
    cube = h5py.File(bio, 'w')
    cube.create_dataset('data', a.shape, compression="gzip", data=a)
    cube.close()

    data = bio.getvalue()
    with fs.open(f'l4dense/neuron-volume/{filename}', 'wb') as f:
        f.write(data)
    return len(data)

In [54]:
fs = gcsfs.GCSFileSystem(project='neuron-jungle', token=credentials)
with fs.open('l4dense/neuron-map.pkl', 'wb') as f:
    f.write(pickle.dumps(neuron_map))

In [ ]:
# x5y8z3 are the largest ids
bytes_total = 0
for i in range(6):
    for j in range(9):
        for k in range(4):
            print(i, j, k)
            bytes_total += repaint(f"x{i}y{j}z{k}.hdf5", credentials)
bytes_total.compute()

In [13]:
fs = gcsfs.GCSFileSystem(project='neuron-jungle', token=credentials)
with open('chunk_template.xdmf', 'r') as f:
    data = f.read()
    
with fs.open('l4dense/chunk_template.xdmf', 'w') as f:
    f.write(data)

In [8]:
np.unique(data.ravel())

array([ 0,  7, 19, 20, 32, 33, 37, 38, 49, 53, 58, 68, 70, 79, 88, 89],
      dtype=uint8)

In [10]:
data = []

In [12]:
!gsutil

/bin/sh: 1: gsutil: not found


In [17]:
import tempfile
import vtk

def fetch_and_cache(filename, credentials, replacement=None):
    fs = gcsfs.GCSFileSystem(project='neuron-jungle', token=credentials)
    
    if replacement is not None:
        mode = 'r'
    else:
        mode = 'rb'
    
    with fs.open(f'l4dense/{filename}', mode) as f:
        data = f.read()
    
    # Write this as a temp file.
    _, filename = tempfile.mkstemp()
    
    if replacement is not None:
        data = data.format(replacement)
    
    if replacement is not None:
        mode = 'w'
    else:
        mode = 'wb'
    
    with open(filename, mode) as f:
        f.write(data)
    
    return filename

def process_one_chunk(filename, credentials):
    index = 7
    xdmf_file = "chunk_template.xdmf"
    local_hdf_file = fetch_and_cache(filename, credentials)
    local_xdmf = fetch_and_cache(xdmf_file, credentials, local_hdf_file)
    
    # Do the 
    colors = vtk.vtkNamedColors()

    # Prepare to read the file.
    readerVolume = vtk.vtkXdmfReader()
    readerVolume.SetFileName(local_xdmf)
    readerVolume.Update()

    # Extract the region of interest.
    voi = vtk.vtkExtractVOI()
    voi.SetInputConnection(readerVolume.GetOutputPort())
    voi.SetVOI(0, 1023, 0, 1023, 0, 1023)
    voi.SetSampleRate(1, 1, 1)
    voi.Update()  # Necessary for GetScalarRange().
    srange = voi.GetOutput().GetScalarRange()  # Needs Update() before!
    print("Range", srange)

    # Prepare surface generation.
    contour = vtk.vtkDiscreteMarchingCubes()  # For label images.
    contour.SetInputConnection(voi.GetOutputPort())
    # contour.ComputeNormalsOn()

    print("Doing label", index)

    contour.SetValue(0, index)
    contour.Update()  # Needed for GetNumberOfPolys()!!!
    
    print("Done contour")

    smoother = vtk.vtkWindowedSincPolyDataFilter()
    smoother.SetInputConnection(contour.GetOutputPort())
    smoother.SetNumberOfIterations(20)  # This has little effect on the error!
    smoother.BoundarySmoothingOff()
    smoother.FeatureEdgeSmoothingOff()
    smoother.SetPassBand(.001)        # This increases the error a lot!
    smoother.NonManifoldSmoothingOn()
    smoother.NormalizeCoordinatesOn()
    smoother.GenerateErrorScalarsOn()
    smoother.Update()

    smoothed_polys = smoother.GetOutput()
    smoother_error = smoothed_polys.GetPointData().GetScalars()

    writer = vtk.vtkXMLDataSetWriter()
    writer.SetFileName("out.vtp")
    writer.SetInputData(smoothed_polys)
    writer.Write()
    
process_one_chunk('neuron-volume/x0y0z0.hdf5', credentials)

ModuleNotFoundError: No module named 'vtkOpenGLKitPython'

We're done!